In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "microsoft/deberta-v3-base"  # DeBERTa 모델
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터 전처리 함수 정의
def preprocess_text(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=128)

# 데이터셋 로드 및 전처리
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['text'], train_data['target'], test_size=0.2, random_state=42
)

# 토큰화 및 변환
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': list(train_labels)
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': list(val_labels)
})

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 모델 학습
trainer.train()

# 검증 데이터 예측
results = trainer.evaluate()
print("Validation Accuracy:", results["eval_accuracy"])

# 테스트 데이터 예측 및 제출 파일 생성
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=128)
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
})

# 예측
predictions = trainer.predict(test_dataset)
test_data['target'] = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

# 제출 파일 생성
submission = test_data[['id', 'target']]
submission.to_csv('submission_v5_DeBERTa_v3.csv', index=False)
print("Submission file saved as 'submission.csv'")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-effde27965d2>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.367400,0.381325,0.835850
2,0.335900,0.432938,0.835850
3,0.371000,0.456798,0.838477


Validation Accuracy: 0.8384767174720764
Submission file saved as 'submission.csv'


In [4]:
# 그리드 서치

from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "microsoft/deberta-v3-base"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)

# 데이터셋 로드
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["text"], train_data["target"], test_size=0.2, random_state=42
)

# 데이터 전처리 함수 정의
def preprocess_data(texts, labels=None):
    encodings = tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
    )
    if labels is not None:
        encodings["labels"] = list(labels)
    return encodings

# 데이터 토큰화
train_encodings = preprocess_data(train_texts, train_labels)
val_encodings = preprocess_data(val_texts, val_labels)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

In [5]:
# 하이퍼파라미터 그리드 정의
grid = {
    "learning_rate": [2e-5, 3e-5, 5e-5],
    "per_device_train_batch_size": [8, 16],
    "num_train_epochs": [2, 3]
}

In [6]:
# 최적의 하이퍼파라미터를 저장할 변수
best_accuracy = 0
best_params = {}

# 그리드 서치 실행
for learning_rate in grid["learning_rate"]:
    for batch_size in grid["per_device_train_batch_size"]:
        for num_epochs in grid["num_train_epochs"]:
            print(f"Testing config: lr={learning_rate}, batch_size={batch_size}, epochs={num_epochs}")

            # 학습 설정
            training_args = TrainingArguments(
                output_dir="./results",
                evaluation_strategy="epoch",
                save_strategy="no",  # 모델 저장 생략
                num_train_epochs=num_epochs,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                learning_rate=learning_rate,
                logging_dir="./logs",
                logging_steps=10,
                report_to="none"
            )

            # 모델 초기화
            model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

            # Trainer 설정
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
            )

            # 모델 학습
            trainer.train()

            # 검증 데이터 평가
            results = trainer.evaluate()
            accuracy = results["eval_accuracy"]
            print(f"Accuracy: {accuracy}")

            # 최적의 하이퍼파라미터 업데이트
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    "learning_rate": learning_rate,
                    "batch_size": batch_size,
                    "num_train_epochs": num_epochs,
                }

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:")
print(best_params)
print(f"Best Validation Accuracy: {best_accuracy}")

Testing config: lr=2e-05, batch_size=8, epochs=2


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-0ad82163c609>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.403900,0.370180,0.848982
2,0.341200,0.434422,0.852922


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.425000,0.413270,0.826001
2,0.303300,0.405053,0.839790


Accuracy: 0.839789867401123
Testing config: lr=5e-05, batch_size=16, epochs=3


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-0ad82163c609>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.420500,0.420178,0.821405
2,0.354500,0.448557,0.821405
3,0.343300,0.449238,0.835850


Accuracy: 0.8358502984046936
Best Hyperparameters:
{'learning_rate': 2e-05, 'batch_size': 8, 'num_train_epochs': 2}
Best Validation Accuracy: 0.8529218435287476


In [7]:
# 최적의 하이퍼파라미터로 모델 재학습
print("Training with Best Hyperparameters...")
best_learning_rate = best_params["learning_rate"]
best_batch_size = best_params["batch_size"]
best_num_epochs = best_params["num_train_epochs"]

# 최적의 하이퍼파라미터로 학습 설정
final_training_args = TrainingArguments(
    output_dir="./final_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=best_num_epochs,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    learning_rate=best_learning_rate,
    logging_dir="./final_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 최종 모델 초기화
final_model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 최종 Trainer 설정
final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 최종 모델 학습
eval_predictions = final_trainer.train()
eval_predictions

Training with Best Hyperparameters...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-dc91b3f4e5f3>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.342700,0.374622,0.847012
2,0.370000,0.448911,0.849639


TrainOutput(global_step=1524, training_loss=0.4167538285255432, metrics={'train_runtime': 162.4475, 'train_samples_per_second': 74.978, 'train_steps_per_second': 9.381, 'total_flos': 569594260806480.0, 'train_loss': 0.4167538285255432, 'epoch': 2.0})

In [8]:
# from sklearn.metrics import classification_report, confusion_matrix
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # 검증 데이터 예측
# ## eval_predictions = final_trainer.predict(val_dataset)

# # 실제 레이블과 예측 레이블 추출
# true_labels = eval_predictions.label_ids
# pred_labels = np.argmax(eval_predictions.predictions, axis=1)

# # 평가지표 계산
# report = classification_report(true_labels, pred_labels, target_names=['no', 'offensive'], output_dict=True)
# report_df = pd.DataFrame(report).transpose()

# # 평가지표 출력
# styled_report = report_df.style.background_gradient(cmap='Blues').format({
#     'precision': '{:.2%}',
#     'recall': '{:.2%}',
#     'f1-score': '{:.2%}',
#     'support': '{:.0f}'
# }).set_caption("Classification Report").set_properties(**{
#     'border': '1px solid black',
#     'color': 'black',
#     'text-align': 'center'
# })
# display(styled_report)

# # 혼동 행렬 계산
# conf_matrix = confusion_matrix(true_labels, pred_labels)

# # 혼동 행렬 시각화
# plt.figure(figsize=(8, 6))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['no', 'offensive'], yticklabels=['no', 'offensive'])
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.title('Confusion Matrix')
# plt.show()

AttributeError: 'TrainOutput' object has no attribute 'label_ids'

In [ ]:
# 테스트 데이터 예측
print("Predicting on Test Data...")
test_encodings = preprocess_data(test_data["text"])
test_dataset = Dataset.from_dict(test_encodings)

test_predictions = final_trainer.predict(test_dataset)
test_data["target"] = torch.argmax(torch.tensor(test_predictions.predictions), axis=1).numpy()

In [10]:
# 제출 파일 생성
submission = test_data[["id", "target"]]
submission.to_csv("submission_v5_DeBERTa_GridSearch.csv", index=False)
print("Submission file saved as 'submission.csv'")

KeyError: "['target'] not in index"